In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))
print("done")


done


# basic model use autogluon
### No EDA

In [2]:
!cp -r /kaggle/input/autogluon-package/* ./

In [3]:
!pip install -q --no-index --find-links='/kaggle/input/autogluon-package/' 'autogluon.tabular-0.7.0-py3-none-any.whl'
print("done")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.133 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.1.2 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
done


In [4]:
!pip install -q ray==2.2.0
print("done")

done


In [5]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [6]:
import random
import os
import pandas as pd
import numpy as np

In [7]:
train = pd.read_csv('../input/playground-series-s3e17/train.csv')
test = pd.read_csv('../input/playground-series-s3e17/test.csv')
submission = pd.read_csv('../input/playground-series-s3e17/sample_submission.csv')

In [8]:
train.describe(include='all')

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429,136429,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
unique,NaN,9976,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,L53257,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,139,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,68214.000000,NaN,NaN,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,NaN,NaN,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,NaN,NaN,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,NaN,NaN,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,NaN,NaN,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,NaN,NaN,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
test.describe(include='all')

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
count,90954.000000,90954,90954,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.00000,90954.000000
unique,NaN,9909,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,L56257,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,98,63438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,181905.500000,NaN,NaN,299.859493,309.939375,1520.528179,40.335191,104.293962,0.001473,0.005343,0.002353,0.00387,0.002309
std,26256.302529,NaN,NaN,1.857562,1.385296,139.970419,8.504683,63.871092,0.038355,0.072903,0.048449,0.06209,0.047995
min,136429.000000,NaN,NaN,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,159167.250000,NaN,NaN,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,181905.500000,NaN,NaN,300.000000,310.000000,1493.000000,40.500000,106.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,204643.750000,NaN,NaN,301.200000,310.900000,1579.000000,46.200000,158.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [10]:
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)
train = train.drop(['Product ID'], axis=1)
test = test.drop(['Product ID'], axis=1)

In [11]:
#this idea by https://www.kaggle.com/code/reymaster/ps3-17-0-96771-eda-feature-engineering-xgboost
train["Power"] = train["Torque [Nm]"] * train["Rotational speed [rpm]"]
train["temp_ratio"] = train["Process temperature [K]"] / train["Air temperature [K]"]
test["Power"] = test["Torque [Nm]"] * train["Rotational speed [rpm]"]
test["temp_ratio"] = test["Process temperature [K]"] / test["Air temperature [K]"]

In [12]:
predictor = TabularPredictor(label='Machine failure', problem_type='binary',
                             eval_metric='roc_auc',sample_weight='auto_weight').fit(train,presets="best_quality",
                                                                    auto_stack=True,save_space=True,
                                                                    num_stack_levels=2, num_bag_folds=2,num_bag_sets=4,time_limit=3600)

# ['accuracy', 'acc', 'balanced_accuracy', 'mcc', 'roc_auc_ovo_macro', 'log_loss', 'nll', 'pac', 'pac_score', 'quadratic_kappa', 'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro',
#  'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted']

No path specified. Models will be saved in: "AutogluonModels/ag-20230613_041028/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=2, num_bag_sets=4
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230613_041028/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Train Data Rows:    136429
Train Data Columns: 13
Label Column: Machine failure
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    32435.92 MB
	Train Data (Original)  Memory Usage: 21.01 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the 

In [13]:
preds = predictor.predict_proba(test)

In [14]:
preds.shape

(90954, 2)

In [15]:
submission.describe()

,id,Machine failure
count,90954.000000,90954.0
mean,181905.500000,0.5
std,26256.302529,0.0
min,136429.000000,0.5
25%,159167.250000,0.5
50%,181905.500000,0.5
75%,204643.750000,0.5
max,227382.000000,0.5


In [16]:
!rm -rf /kaggle/working/*

In [17]:
submission['Machine failure'] = preds.iloc[: , 1]
submission.to_csv('/kaggle/working/submission.csv', index=False)